## Importer av paket och data från REMbox

In [ ]:
import pandas as pd 
import math
import hvplot.pandas #noqa #plotpaket
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from rembox_integration_tools import REMboxDataQuery
from rembox_integration_tools.rembox_analysis import StudyColumn, SeriesColumn
from pathlib import Path

# om plotly önskas så skrivs följande hvplot.extension("plotly")
hvplot.extension("bokeh")

CLIENT_ID_ENV_VAR = "REMBOX_INT_CLIENT_ID"
CLIENT_PWD_ENV_VAR = "REMBOX_INT_CLIENT_PWD"
TOKEN_URI = "https://autoqa.vll.se/dpqaauth/connect/token" #Var finns access token
API_URI = "https://rembox.vll.se/api" #Var finns API:t
ORIGIN_URI = "https://rembox.vll.se" #Vilken URL

rembox = REMboxDataQuery(
    client_id_environment_variable=CLIENT_ID_ENV_VAR,
    client_secret_environment_variable=CLIENT_PWD_ENV_VAR,
    token_uri=TOKEN_URI,
    api_uri=API_URI,
    origin_uri=ORIGIN_URI,
    verify_ssl_cert=False
)

valid_study_columns = StudyColumn()
valid_series_columns = SeriesColumn()

In [ ]:
rembox.reset_filter_options()
def get_data_from_fluoro(rembox: REMboxDataQuery) -> tuple[pd.DataFrame, pd.DataFrame]:

    
    rembox.filter_options.set_inclusive_tags(
        machine_types=["XASTAT"],     # CT-CT, Fluoroscopic-XASTAT, Mobile C-arm-XAMOB, Conventional-DX, Mammography-MG, Intraoral-IO, Panoramic-PX, Dental Cone Beam CT-DCBCT, PET-PET, PET/CT-PETCT, SPECT-SPECT, SPECT/CT-SPECTCT, Nuclear Medicine-NM, Mobile X-ray-DXMOB, Conventional with fluoro-DXXA
        machines=["U104", "U104_2024", "U104_2023"]#, "U105", "U105_2022_05", "U106", "U106_2023_01", "U601", "U602", "Arytmi1", "Arytmi2", "Arytmi2_2024"]        # INR, IR1, IR2, PCI1, PCI2, Morran, Mumin
    )
    
    #rembox.filter_options.set_exclusive_tags() om jag vill ange filter där man bortser från ett visst kriterie

    rembox.filter_options.patient_age_interval_include_nulls = True
    
    rembox.filter_options.study_time_interval_start_date = "2022-01-01T00:00:00Z"
    rembox.filter_options.study_time_interval_end_date = "2025-04-08T00:00:00Z"
    
    rembox.deanonymize_performing_physician = True
    
    rembox.add_columns(
        columns=[
            valid_study_columns.StudyDateTime,
            valid_study_columns.AccessionNumber,
            valid_study_columns.AcquisitionDoseAreaProductTotal,
            valid_study_columns.AcquisitionDoseRPTotal,
            valid_study_columns.AcquisitionPlane,
            #valid_study_columns.CalibrationDate,
            #valid_study_columns.CalibrationFactor,
            #valid_study_columns.CalibrationProtocol,
            #valid_study_columns.CalibrationResponsibleParty,
            #valid_study_columns.CalibrationUncertainty,
            valid_study_columns.City,
            valid_study_columns.ConvFluoroClassifier,
            valid_study_columns.DoseAreaProductTotal,
            valid_study_columns.DoseMeasurementDevice,
            valid_study_columns.DoseRPTotal,
            valid_study_columns.FluoroDoseAreaProductTotal,
            valid_study_columns.FluoroDoseRPTotal,
            valid_study_columns.HasIntent,
            #valid_study_columns.HalfValueLayer,
            valid_study_columns.Hospital,
            valid_study_columns.Id,
            valid_study_columns.Machine,
            #valid_study_columns.MeanBodyThickness,
            #valid_study_columns.MaximumBodyThickness,
            #valid_study_columns.MinimumBodyThickness,
            valid_study_columns.PatientAge,
            valid_study_columns.PatientAgeUnit,
            valid_study_columns.PatientDbId,
            valid_study_columns.PatientId,
            #valid_study_columns.PatientModel,
            #valid_study_columns.PatientsBodyMassIndex,
            #valid_study_columns.PatientsName,
            valid_study_columns.PatientsSex,
            valid_study_columns.PatientsSize,
            valid_study_columns.PatientsSizeDate,
            valid_study_columns.PatientsSizeSource,
            valid_study_columns.PatientsWeight,
            valid_study_columns.PatientsWeightDate,
            valid_study_columns.PatientsWeightSource,
            #valid_study_columns.PerformingPhysicianIdentificationSequence,
            valid_study_columns.PSD, # ------------------------------------------------------PSD?
            valid_study_columns.PerformingPhysicianName,
            #valid_study_columns.PregnancyStatus,
            valid_study_columns.ProcedureCode,
            valid_study_columns.ProcedureCodeMeaning,
            valid_study_columns.ProcedureReported,
            valid_study_columns.ProtocolCode,
            valid_study_columns.ProtocolCodeMeaning,
            #valid_study_columns.ReferenceAuthority,
            #valid_study_columns.ReferencedSopInstanceUid,
            valid_study_columns.ReferencePointDefinition,
            #valid_study_columns.ReferencePointDefinitionCode,
            valid_study_columns.ReferringPhysicianIdentificationSequence,
            valid_study_columns.ReferringPhysiciansName,
            #valid_study_columns.RequestedProcedureCode,
            #valid_study_columns.RequestedProcedureCodeMeaning,
            valid_study_columns.ScopeOfAccumulation,
            valid_study_columns.SoftwareVersions,
            #valid_study_columns.StartOfXrayIrradiation,
            #valid_study_columns.StudyDateTime, --------- La denna överst
            valid_study_columns.StudyDescription,
            valid_study_columns.StudyId,
            valid_study_columns.StudyInstanceUID,
            valid_study_columns.TotalAcquisitionTime,
            valid_study_columns.TotalFluoroTime,
            valid_study_columns.TotalNumberOfIrradiationEvents,
            valid_study_columns.TotalNumberOfRadiographicFrames,
            valid_series_columns.AcquisitionPlaneSeries,
            valid_series_columns.AcquisitionProtocol,
            #valid_series_columns.AcquisitionType,
            #valid_series_columns.ApplicationName, #--------------------------------------Här finns protokollnamn för Azurion
            #valid_series_columns.AnatomicalStructure,
            #valid_series_columns.AnodeTargetMaterial,
            valid_series_columns.AverageXrayTubeCurrent,
            valid_series_columns.CollimatedFieldArea,
            valid_series_columns.CollimatedFieldHeight,
            valid_series_columns.CollimatedFieldWidth,
            #valid_series_columns.ColumnAngulation,
            #valid_series_columns.CrdrMechanicalConfiguration,
            valid_series_columns.DateTimeStarted,
            #valid_series_columns.DerivedEffectiveDiameter,
            #valid_series_columns.DeviationIndex,
            valid_series_columns.DistanceSourceToDetector,
            valid_series_columns.DistanceSourceToIsocenter,
            valid_series_columns.DistanceSourceToReferencePoint,
            #valid_series_columns.DistanceSourceToTablePlane,
            valid_series_columns.DoseAreaProduct,
            valid_series_columns.DoseRP,
            #valid_series_columns.EffectiveDose,
            #valid_series_columns.EffectiveDoseConversionFactor,
            #valid_series_columns.EntranceExposureAtRP,
            #valid_series_columns.ExposedRange,
            valid_series_columns.Exposure,
            #valid_series_columns.ExposureIndex,
            #valid_series_columns.ExposureTime,
            #valid_series_columns.ExposureTimePerRotation,
            valid_series_columns.FluoroMode,
            #valid_series_columns.FluoroFlavour, # -------------------------------------------- Här finns pulsrat för Azurion
            #valid_series_columns.FrameOfReferenceUID,
            #valid_series_columns.IdentificationOfTheXraySource,
            #valid_series_columns.ImageView,
            #valid_series_columns.ImageViewModifier,
            valid_series_columns.IrradiationDuration,
            #valid_series_columns.IrradiationEventLabel,
            valid_series_columns.IrradiationEventType,
            valid_series_columns.IrradiationEventUID,
            valid_series_columns.kVp,
            #valid_series_columns.LabelType,
            #valid_series_columns.Laterality,
            #valid_series_columns.MaximumXrayTubeCurrent,
            #valid_series_columns.MeasurementMethodDose,
            #valid_series_columns.NominalCollimationWidth,
            #valid_series_columns.NominalTotalCollimationWidth,
            valid_series_columns.NumberOfPulses,
            #valid_series_columns.NumberOfXraySources,
            valid_series_columns.PatientEquivalentThickness,
            valid_series_columns.PatientOrientation,
            valid_series_columns.PatientOrientationModifier,
            valid_series_columns.PatientTableRelationship,
            valid_series_columns.PositionerPrimaryAngle,
            #valid_series_columns.PositionerPrimaryEndAngle,
            valid_series_columns.PositionerSecondaryAngle,
            #valid_series_columns.PositionerSecondaryEndAngle,
            #valid_series_columns.ProcedureContext,
            #valid_series_columns.ProjectionEponymousName,
            valid_series_columns.PulseRate,
            valid_series_columns.PulseWidth,
            #valid_series_columns.ReconstructionAlgortihm,
            valid_series_columns.ReferencePointDefinitionText,
            valid_series_columns.SpotSize,
            valid_series_columns.TableCradleTiltAngle,
            valid_series_columns.TableHeadTiltAngle,
            #valid_series_columns.TableHeightEndPosition,
            valid_series_columns.TableHeightPosition,
            valid_series_columns.TableHorizontalRotationAngle,
            #valid_series_columns.TableLateralEndPosition,
            valid_series_columns.TableLateralPosition,
            #valid_series_columns.TableLongitudinalEndPosition,
            valid_series_columns.TableLongitudinalPosition,
            #valid_series_columns.TargetExposureIndex,
            valid_series_columns.TargetRegion,
            #valid_series_columns.WaterEquivalentDiameter,
            #valid_series_columns.WedMeasurementMethod,
            #valid_series_columns.XrayFilterAluminumEquivalent,
            valid_series_columns.XrayFilterMaterial,
            valid_series_columns.XrayFilterThicknessMaximum,
            valid_series_columns.XrayFilterThicknessMinimum,
            valid_series_columns.XrayFilterType,
            #valid_series_columns.XrayGrid,
            #valid_series_columns.XrayGridAspectRatio,
            #valid_series_columns.XrayGridFocalDistance,
            #valid_series_columns.XrayGridPitch,
            #valid_series_columns.XrayModulationType,
            valid_series_columns.XrayTubeCurrent
        ]
    )

    return rembox.run_query()

In [ ]:
#Hämta data från REMbox
study_data, series_data = get_data_from_fluoro(rembox=rembox)

In [ ]:
#Export av data till csv
study_data.to_csv("C:/Users/chgr09/GIT/rvbrtg/Data/output_data/XA_study_2023.csv")
#series_data.to_csv("C:/Users/chgr09/GIT/rvbrtg/Data/output_data/XA_series_2023.csv")

## Kontroller av data och hantering av dataframes

In [ ]:
study = study_data.copy() #skapa kopia av dataframe på study-nivå för att kunna behålla orginalet
series = series_data.copy() #skapa kopia av dataframe på serie-nivå för att kunna behålla orginalet

#Ta bort undersökningar från testpatient U105 som har PatientId = LO_sgY+/xUXOYtGGM1+o+JXSO9IQy9LaYFpsPIl/UAWtP0=
#Ta bort undersökningar från testpatient U106 som har PatientId = LO_9a9c67eb_5dc1_43ed_8008_858c4683e27d

In [ ]:
#Räkna antal ingrepp och antal unika patienter
exams = study["patientDbId"].count()
patients = study["patientDbId"].nunique() #Count number of distinct elements in specified axis. Can ignore NaN values

print(exams, "undersökningar/ingrepp fördelat på", patients, "patienter")

# Ta bort alla dubletter där 1mm Al-filter visas istället för Cu-filter 
# TODO: Gör separata kolumner för Al och Cu-filter så det inte blir dubbla rader.

series = series[(series['xrayFilterMaterial'] == 'Copper') | (series['xrayFilterMaterial'] == 'Copper or Copper compound')]
print(len(series), "antal irradiation events")

## Addering av events för att få med båda planen. Viktigt för Biplan

In [ ]:
#Denna behövs inte längre då vi nu får rader med både Plan A och Plan B i samma dataframe. Kan eventuellt behövas för att joina in Rotational Acquisition senare.

plane_A_doses = (series[series['acquisitionPlaneSeries'] == 'Plane A']
                .groupby(["studyInstanceUID", "irradiationEventType"])
                .agg({'doseAreaProduct': 'sum', 'doseRP': 'sum', 'irradiationDuration': 'sum'})
                .reset_index().round(3)) 
plane_A_doses.rename(columns={"doseAreaProduct": "doseAreaProductPlaneA", "doseRP": "doseRpPlaneA", "irradiationDuration": "irradiationDurationPlaneA"}, inplace=True)

plane_B_doses = (series[series['acquisitionPlaneSeries'] == 'Plane B']
                .groupby(["studyInstanceUID", "irradiationEventType"])
                .agg({'doseAreaProduct': 'sum', 'doseRP': 'sum', 'irradiationDuration': 'sum'})
                .reset_index().round(3)) 
plane_B_doses.rename(columns={"doseAreaProduct": "doseAreaProductPlaneB", "doseRP": "doseRpPlaneB", "irradiationDuration": "irradiationDurationPlaneB"}, inplace=True)

Plane_A_and_B_doses = plane_A_doses.merge(plane_B_doses, on = ['studyInstanceUID', 'irradiationEventType'], how = "left")

#study_AB = study.merge(Plane_A_and_B_doses, on = ['studyInstanceUID'], how = "left")

Plane_A_and_B_doses.head()

In [ ]:
# Filter the series DataFrame for Plane A and Rotational Acquisition
# TODO: Utveckla detta med ackumuerade värden för respektive protokoll som FL Låg, Exponering och CBCT kontra 3DRA

rotational_acquisition_plane_a = series[
    (series['acquisitionPlaneSeries'] == 'Plane A') & 
    (series['irradiationEventType'] == 'Rotational Acquisition')
]

# Group by studyInstanceUID and sum the DoseAreaProduct
rotational_dap_plane_a = rotational_acquisition_plane_a.groupby('studyInstanceUID')['doseAreaProduct'].sum().reset_index()

# Rename the column for clarity
rotational_dap_plane_a.rename(columns={'doseAreaProduct': 'accumulatedRotationalDapPlaneA'}, inplace=True)

# Merge the accumulated Rotational DoseAreaProduct into the study DataFrame
study = study.merge(rotational_dap_plane_a, on='studyInstanceUID', how='left')

# Fill NaN values with 0 (if there are studies without Rotational Acquisition for Plane A)
study['accumulatedRotationalDapPlaneA'] = study['accumulatedRotationalDapPlaneA'].fillna(0)

# Display the updated study DataFrame
#study.head()

In [ ]:
#Ta bort felaktiga värden för Rot Acq DAP (endast Plan A ska ha värden)
study.loc[study['acquisitionPlane'] == 'Plane B', 'accumulatedRotationalDapPlaneA'] = 0
#Räkna ut den andel av DAP som inte kommer från Rot Acq
study['acquisitionDoseAreaProductTotalminusRotational'] = study['acquisitionDoseAreaProductTotal'] - study['accumulatedRotationalDapPlaneA']

# Normalisera DAP till gmltid
study['normalisedFluoroDap'] = study['fluoroDoseAreaProductTotal'] / study['totalFluoroTime']
study['normalisedAcqDap'] = study['acquisitionDoseAreaProductTotal'] / study['totalAcquisitionTime']
study['normalisedFluoroRP'] = study['fluoroDoseRPTotal'] / study['totalFluoroTime']
study['normalisedAcqRP'] = study['acquisitionDoseRPTotal'] / study['totalAcquisitionTime']


## Lägg till operatörsnamn

In [ ]:
# Översättningstabell från pseudo-operatörer till operatörer
names_data_path = "C:/Projekt/GIT/rvbrtg/Data/input_data/operators_2025.xlsx" 

names = pd.read_excel(names_data_path)

#Ändra kolumnnamn för att kunna göra en join
names.columns = ["performingPhysicianName", "operatorName"]
#names.head()

#Joina dataframes för att få in operatörsnamn
study = study.merge(names, on = ["performingPhysicianName"], how = "left")

#Print för att kolla så att det funkade
#print(study.operatorName)

#Granska om namn saknas
#study_names.to_csv("C:/Projekt/GIT/rvbrtg/Data/output_data/Operators_missing_GML_2024.csv")

In [ ]:
#Granska om namn saknas
study[study['machine'] == 'U104'].to_csv("C:/Projekt/GIT/rvbrtg/Data/output_data/Operators_missing_INR_2025.csv")

In [ ]:
study.head()

In [ ]:
#Alternativa y-värden: fluoroDoseAreaProductTotal  acquisitionDoseAreaProductTotal  accumulatedRotationalDapPlaneA doseRPTotal acquisitionDoseAreaProductTotalminusRotational  doseAreaProductTotal

fig = px.scatter(study[study['acquisitionPlane'] == 'Plane A'], x="studyDateTime", y="doseRPTotal", color="procedureCodeMeaning")#, symbol="procedureCodeMeaning")
#fig.update_layout(height=1000)
fig.show()

In [ ]:
# Tabell över ackumulerad DAP, total DAP och antal ingrepp per Kod.

procedure_dap = (study.groupby(["procedureCode", "procedureCodeMeaning", "machine", "acquisitionPlane"])
                    .agg({'doseAreaProductTotal': ['sum', 'median'], 'fluoroDoseAreaProductTotal': 'median', 'totalFluoroTime': 'median', 'normalisedFluoroDap': 'median', 'procedureCode': 'count'}
                         ).reset_index().round(3))
procedure_dap.head()

In [ ]:
# Tabell över ackumulerad DAP, total DAP och antal ingrepp per Kod.

procedure_dap = (study.groupby(["procedureCode", "procedureCodeMeaning", "machine", "acquisitionPlane", "operatorName"])
                    .agg({'doseAreaProductTotal': ['count', 'median'], 'acquisitionDoseAreaProductTotal': 'median', 'acquisitionDoseAreaProductTotalminusRotational': 'median','accumulatedRotationalDapPlaneA': 'median', 'fluoroDoseAreaProductTotal': 'median', 'totalFluoroTime': 'median', 'normalisedFluoroDap': 'median'}
                         ).reset_index().round(3))
procedure_dap.head()

In [ ]:
removals = study['procedureCodeMeaning'].value_counts().reset_index()
removals = removals[removals['count'] > 10] # Ändra tröskel för hur många undersökningar som måste finnas med samma Kod i dataframe
removals

filtered_df = study[study['procedureCodeMeaning'].isin(removals['procedureCodeMeaning'])]
#filtered_df.head()

procedure_dap = (filtered_df[(filtered_df['machine'] == 'U104') & (filtered_df['acquisitionPlane'] == 'Plane A')].groupby(["procedureCode", "procedureCodeMeaning", "machine"]) #, "acquisitionPlane", "operatorName"
                    .agg({'doseAreaProductTotal': ['count', 'median'], 'acquisitionDoseAreaProductTotal': 'median', 'acquisitionDoseAreaProductTotalminusRotational': 'median', 'accumulatedRotationalDapPlaneA': 'median', 'fluoroDoseAreaProductTotal': 'median', 'totalFluoroTime': 'median', 'normalisedFluoroDap': 'median'}
                         ).reset_index().round(3))
procedure_dap.head()

In [ ]:
fig = px.box(study[study['acquisitionPlane'] == 'Plane A'], x='procedureCodeMeaning', y='doseAreaProductTotal', color='machine')
#fig.update_layout(height=1000)
fig.show()

In [ ]:
# Filter the data for machine U104
study_U104 = study[study['machine'] == 'U104']

# Filter procedure codes with sufficient occurrences
removals = study_U104['procedureCodeMeaning'].value_counts().reset_index()
removals = removals[removals['count'] > 10]  # Adjust threshold for filtering
filtered_df = study_U104[study_U104['procedureCodeMeaning'].isin(removals['procedureCodeMeaning'])]

# Get unique procedureCode values
unique_procedure_codes = filtered_df['procedureCode'].unique()

# Calculate the number of rows and columns for a symmetric layout
num_procedures = len(unique_procedure_codes)
grid_size = math.ceil(math.sqrt(num_procedures))  # Number of rows and columns

# Calculate median doseAreaProductTotal for each procedureCode
median_dap = filtered_df.groupby('procedureCode')['doseAreaProductTotal'].median()

# Create a subplot layout with a symmetric grid
fig = make_subplots(
    rows=grid_size,
    cols=grid_size,
    specs=[[{'type': 'domain'} for _ in range(grid_size)] for _ in range(grid_size)],
    subplot_titles=[
        f"ProcedureCode {code}<br>Median DAP: {median_dap[code]:.2f}" if code in median_dap else f"ProcedureCode {code}"
        for code in unique_procedure_codes
    ]
)

# Loop through each procedureCode and create a pie chart
for idx, code in enumerate(unique_procedure_codes):
    # Calculate the row and column index
    row = (idx // grid_size) + 1
    col = (idx % grid_size) + 1

    # Filter the data for the current procedureCode and Plane A
    procedure_data = filtered_df[(filtered_df['procedureCode'] == code) & 
                                 (filtered_df['acquisitionPlane'] == 'Plane A')]

    # Extract the median values for the required columns
    acquisition_dap = procedure_data['acquisitionDoseAreaProductTotalminusRotational'].median()
    fluoro_dap = procedure_data['fluoroDoseAreaProductTotal'].median()
    rotational_dap = procedure_data['accumulatedRotationalDapPlaneA'].median()

    # Combine the values into a list
    values = [acquisition_dap, fluoro_dap, rotational_dap]
    labels = ['Acquisition DAP', 'Fluoro DAP', 'Rotational DAP']

    # Add the pie chart to the subplot
    fig.add_trace(
        go.Pie(labels=labels, values=values, name=f"ProcedureCode {code}"),
        row=row,
        col=col
    )

# Update layout
fig.update_layout(
    height=300 * grid_size,  # Adjust height based on the grid size
    width=300 * grid_size,   # Adjust width based on the grid size
    title_text="Median DAP Distribution for All ProcedureCodes (Plane A)",
    showlegend=True  # Disable global legend for cleaner layout
)

# Show the figure
fig.show()

In [ ]:
# Filter the data for machine U104
study_U104 = study[study['machine'] == 'U104']

# Filter procedure codes with sufficient occurrences
removals = study_U104['procedureCodeMeaning'].value_counts().reset_index()
removals = removals[removals['count'] > 10]  # Adjust threshold for filtering
filtered_df = study_U104[study_U104['procedureCodeMeaning'].isin(removals['procedureCodeMeaning'])]

# Get unique procedureCodeMeaning values
unique_procedure_codes = filtered_df['procedureCodeMeaning'].unique()

# Calculate the number of rows and columns for a symmetric layout
num_procedures = len(unique_procedure_codes)
grid_size = math.ceil(math.sqrt(num_procedures))  # Number of rows and columns

# Calculate median doseAreaProductTotal for each procedureCodeMeaning
median_dap = filtered_df.groupby('procedureCodeMeaning')['doseAreaProductTotal'].median()

# Create a subplot layout with a symmetric grid
fig = make_subplots(
    rows=grid_size,
    cols=grid_size,
    specs=[[{'type': 'domain'} for _ in range(grid_size)] for _ in range(grid_size)],
    subplot_titles=[
        f"ProcedureCodeMeaning<br> {code}<br>Median DAP: {median_dap[code]:.2f}" if code in median_dap else f"ProcedureCodeMeaning {code}"
        for code in unique_procedure_codes
    ]
)

# Loop through each procedureCodeMeaning and create a pie chart
for idx, code in enumerate(unique_procedure_codes):
    # Calculate the row and column index
    row = (idx // grid_size) + 1
    col = (idx % grid_size) + 1

    # Filter the data for the current procedureCodeMeaning and Plane A
    procedure_data = filtered_df[(filtered_df['procedureCodeMeaning'] == code) & 
                                 (filtered_df['acquisitionPlane'] == 'Plane A')]

    # Extract the median values for the required columns
    acquisition_dap = procedure_data['acquisitionDoseAreaProductTotalminusRotational'].median()
    fluoro_dap = procedure_data['fluoroDoseAreaProductTotal'].median()
    rotational_dap = procedure_data['accumulatedRotationalDapPlaneA'].median()

    # Combine the values into a list
    values = [acquisition_dap, fluoro_dap, rotational_dap]
    labels = ['Acquisition DAP', 'Fluoro DAP', 'Rotational DAP']

    # Add the pie chart to the subplot
    fig.add_trace(
        go.Pie(labels=labels, values=values, name=f"ProcedureCodeMeaning {code}"),
        row=row,
        col=col
    )

# Update layout
fig.update_layout(
    height=300 * grid_size,  # Adjust height based on the grid size
    width=300 * grid_size,   # Adjust width based on the grid size
    #title_text="Median DAP Distribution for All ProcedureCodes (Plane A)",
    showlegend=True  # Disable global legend for cleaner layout
)

# Show the figure
fig.show()

In [ ]:
median_KAP_studytype = (study[study['acquisitionPlane'] == 'Plane A'].groupby(
    ["procedureCode", "procedureCodeMeaning", "operatorName", "machine"])
                           .agg({'doseAreaProductTotal': ['min', 'median', 'max'], 'normalisedFluoroDap': 'median', 'procedureCode': 'count'}
                                 ).reset_index().round(3)) 
median_KAP_studytype

In [ ]:
removals = study['procedureCodeMeaning'].value_counts().reset_index()
removals = removals[removals['count'] > 10] # Ändra tröskel för hur många undersökningar som måste finnas med samma Kod i dataframe
removals

filtered_df = study[study['procedureCodeMeaning'].isin(removals['procedureCodeMeaning'])]
#filtered_df.head()

fig = px.box(filtered_df[filtered_df['acquisitionPlane'] == 'Plane A'], x='procedureCodeMeaning', y='doseAreaProductTotal', color='machine', points='all')
fig.update_layout(height=1000)
fig.show()

In [ ]:
fig = px.box(series[series.irradiationEventType == 'Fluoroscopy'], x='acquisitionProtocol', y='doseAreaProduct')#, color='acquisitionProtocol')  
#fig.update_layout(height=800)  
fig.show()

In [ ]:
fig = px.box(study[study['acquisitionPlane'] == 'Plane A'], x='protocolCodeMeaning', y='doseAreaProductTotal', color='operatorName')  
#fig.update_layout(height=800)  
fig.show()

In [ ]:
#Join mellan study_names och series för att addera summerad doseRP
total_doseRP_study_and_plane = series.groupby(
    ["studyInstanceUID", "accessionNumber", "acquisitionPlaneSeries"]).sum("doseRP").reset_index()
total_doseRP_study_and_plane.head()
#study_names_planes = study_names.merge(total_doseRP_study_and_plane, on = ["studyInstanceUID", "accessionNumber"], how = "left")
#study_names_planes.head()

In [ ]:
removals = study_AB['procedureCodeMeaning'].value_counts().reset_index()
removals = removals[removals['count'] > 10] # Ändra tröskel för hur många undersökningar som måste finnas med samma Kod i dataframe
removals

filtered_df_planes = study_AB[study_AB['procedureCodeMeaning'].isin(removals['procedureCodeMeaning'])]
#filtered_df.head()

fig = go.Figure()

for machine in study_AB['machine'].unique():
    machine_data = study_AB[study_AB['machine'] == machine]
    
    fig.add_trace(go.Box(
        y=machine_data['doseAreaProduct_A'],
        x=machine_data['procedureCodeMeaning'],
        name=f'PlaneA - {machine}'
    ))
    fig.add_trace(go.Box(
        y=machine_data['doseAreaProduct_B'],
        x=machine_data['procedureCodeMeaning'],
        name=f'PlaneB - {machine}'
    ))

fig.update_layout(boxmode='group')
fig.show()


In [ ]:
fig = px.scatter(study_AB, x="studyDateTime", y="doseAreaProduct_A", color="machine")

fig.show()

In [ ]:
#Filtrera fram alla ingrepp där summerade doseRP från study_names_planes överstiger 5 Gy

study_5Gy_names = study_names_planes[study_names_planes.doseRP > 5000]

# Kika på innehåll i dataframe
study_5Gy_names[["accessionNumber","studyDateTime","studyDescription","machine","doseRPTotal","acquisitionPlaneSeries","doseRP","OperatorName"]]

In [ ]:
#Testa dumpa ut en undersökning till Excel för att granska vidare

#series_data[series_data.accessionNumber == "SERUME0007509112" & pd.isna(series_data.acquisitionPlaneSeries)].doseRP
test = series_data[series_data.accessionNumber == "SERUME0007751954"]
test.to_excel("C:/Users/chgr09/GIT/rvbrtg/Data/output_data/Binjure.xlsx")

## Undersökningar som överstigit 5Gy

In [ ]:
#Filtrera fram alla ingrepp där doseRPTotal överstiger 5 Gy

study_5Gy = study[study.doseRPTotal > 5000]

# Kika på innehåll i dataframe
study_5Gy

In [ ]:
# Skriv till Excel
study_5Gy.to_excel("output_data/Över_5Gy.xlsx")

# Multiplar. Tester nedan.

In [ ]:
multiple = study.groupby("patientDbId", as_index=False)["doseRPTotal"].nunique()
multiple

In [ ]:
test = study_data.groupby(["patientDbId"])["doseRPTotal"].sum().reset_index()
test.drop(test[(test["doseRPTotal"] <5000)].index, inplace=True)
test

In [ ]:
top_pat = study_data[study_data.patientDbId == 84305]
top_pat

In [ ]:
def loop_analysis(data: pd.DataFrame, start_date: datetime, end_date: datetime) -> pd.DataFrame:
    analysis_data = data[(data.studyDateTime >= start_date & data.studyDateTime <= end_date)]

    test = analysis_data.groupby(
        ["patientDbId", "acquisitionPlaneSeries"]).sum(
        "doseRP").reset_index()
    test = test[["patientDbId", "acquisitionPlaneSeries", "doseRP", "studyDateTime"]]

    return test[test.doseRP > 5000]

In [ ]:
from dateutil.relativedelta import relativedelta
new_study_data = study.copy()
new_study_data.studyDateTime = pd.to_datetime(new_study_data.studyDateTime, infer_datetime_format=True)
new_study_data["studyDate"] = pd.to_datetime(new_study_data.studyDateTime.dt.date)

for end_date in sorted(new_study_data.studyDate.unique().tolist()):
    sd = pd.to_datetime(datetime.fromtimestamp(end_date / 1e9) - relativedelta(months=3))
    resultat = loop_analysis(new_study_data, start_date=sd, end_date=end_date)

test = new_study_data.groupby(["patientDbId", "acquisitionPlaneSeries", pd.Grouper(key="studyDateTime", freq="12M")]).sum("doseRP").reset_index()
test = test[["patientDbId", "acquisitionPlaneSeries", "doseRP", "studyDateTime"]]
study_data_5Gy = test[test.doseRP > 5000]